# LLM Fine-Tuning for Financial Question Answering
Baseline vs LoRA Comparison


In [ ]:
!pip install -q transformers datasets peft accelerate evaluate


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import evaluate


## Load Dataset


In [ ]:
dataset = load_dataset("ExplodingGradients/fiqa", "main")
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']


## Load Base Model (DistilGPT2)


In [ ]:
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)


## Baseline Evaluation


In [ ]:
def generate_answer(question):
    inputs = tokenizer(question, return_tensors="pt", truncation=True, padding=True)
    outputs = model.generate(**inputs, max_length=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


## Apply LoRA


In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


## Training Setup


In [ ]:
def tokenize_function(example):
    text = example['question'] + " " + example['ground_truths'][0]
    return tokenizer(text, truncation=True, padding='max_length', max_length=128)

small_train = train_dataset.select(range(1000)).map(tokenize_function)
small_val = val_dataset.select(range(200)).map(tokenize_function)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_val
)


## Train Model


In [ ]:
trainer.train()


## Final Comparison
Baseline F1: 0.1189
LoRA Fine-Tuned F1: 0.0488
